
# Prompt Engineering Lab on Google AI Platform (Vertex AI)

> **Note:** This lab should be done after the intro_prompt_design.ipynb where you will setup your VertexAI credentials.

**First Colab**: <table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/prompts/intro_prompt_design.ipynb#scrollTo=154137022fb6" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>




In [83]:
!pip install -U google-genai python-dotenv


### Environment configuration

In [84]:

import os
from google import genai
from google.genai import types
from IPython.display import Markdown, display
from google.genai.types import GenerateContentConfig

# --- Set your project and region here ---
PROJECT_ID = "xenon-notch-443912-b5"
LOCATION   = "us-central1"

client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print("Vertex client ready:", client is not None)
print("Project:", PROJECT_ID, "Location:", LOCATION)


Vertex client ready: True
Project: xenon-notch-443912-b5 Location: us-central1



### Choose a model

We’ll use **Gemini 2.5** family.  
- **`gemini-2.5-pro`**: best reasoning (higher cost/latency).  
- **`gemini-2.5-flash`**: fast & economical (great for labs).

We'll also define helpers for token counting and safe printing.


In [85]:

MODEL_FAST = "gemini-2.5-flash"
MODEL_PRO  = "gemini-2.5-pro"

def count_tokens(prompt_or_contents, model=MODEL_FAST):
    """Return token usage estimate for a given input.    Works with Vertex AI per google-genai docs."""
    resp = client.models.count_tokens(model=model, contents=prompt_or_contents)
    return resp.total_tokens

def pp(resp):
    """Pretty-print text from a generate_content response."""
    try:
        print(resp.text)
    except Exception as e:
        print("No .text found, raw response:")
        print(resp)

print("Models ready.")


Models ready.


--------

## 1. Prompt Pattern
#### A- Persona Pattern : Act as persona X, Perform task Y
Write a prompt example where you use the persona pattern to the LLM

In [86]:
from google.colab import auth
auth.authenticate_user()

In [87]:
# TODO: Persona Pattern
prompt="""Act like cheerful tourist guide and answer the question

question=Which is the best place to visit in France other than Paris?
answer:
"""

response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

Bonjour, lovely adventurers! Oh là là, what a wonderful question! While Paris certainly steals many hearts, France is absolutely bursting with other magnificent treasures just waiting to be discovered!

If you're asking me for *the* best place beyond the City of Lights, my dear friends, I simply *have* to point you towards the utterly enchanting region of **Provence**! ✨

Imagine this: rolling lavender fields stretching as far as the eye can see, charming hilltop villages straight out of a postcard, the scent of wild herbs in the air, and sunshine, glorious sunshine almost all year round! It's like stepping into a painting by Van Gogh or Cézanne himself!

You could wander through the historic Pope's Palace in Avignon, sip a delightful rosé in a sun-drenched café in Aix-en-Provence, explore the vibrant markets brimming with local produce and artisanal crafts, or get lost in the ochre-colored streets of Roussillon and the breathtaking views from Gordes. The food, the wine, the history, the incredible light – it's all just *magnifique*!

It's a place that truly soothes the soul and delights all your senses. Trust me, a trip to Provence will fill your heart with joy and leave you with memories as beautiful as its landscapes! *C'est parfait!* So, when are we going?!

#### B- Audience Persona Pattern : Assume that I am Persona X, explain to me Y
Write a prompt example where you use the Audience persona pattern to the LLM

In [88]:
# TODO: Audience Persona Pattern
prompt="""Act as if you are talking to a group of fans

example1:
Question1:What is your next movie?
Answer:My next movie is ABCD chapter 2

example2:What are your current projects?
Answer: As of now, I'm doing 2 movies in Malayalam, 1 in Hindi and 2 in Kannada.

example3:What genre will be the next movie?
Answer:
"""

response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

Oh, that's a great question! For my next one, we're diving deep into an **action-thriller** with a strong emotional core. Think high stakes, intense sequences, and a story that'll keep you guessing until the very end. I can't wait for you all to see it!

#### C- Question Refinement Pattern : Ensure that LLM suggests potentially better or more refined questions the user could ask instead of original one.
**Example**: Whenever I ask a question, suggest a better question and ask me if I would like to use it instead.
What I can do next weekend to be happy?

Write a prompt example where you use the Question Refinement Patternto the LLM

In [89]:
#TODO: Question Refinement Pattern
prompt="""Suggest me some better question patterns whenever I ask a question and ask if I would like to use it instead

question=Can we watch a movie today or would a concert be better?
answer:
"""

response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

That's a great question, and it highlights a common scenario where a slight rephrasing can make a big difference in how the other person responds!

The original question, "Can we watch a movie today or would a concert be better?" is perfectly understandable, but it can feel a little bit like a test or put the other person on the spot to pick the "better" option.

Here are a few patterns that can make your questions more engaging, collaborative, and open to broader responses, applied to your example:

---

### **Suggested Question Patterns:**

1.  **The Collaborative & Open-Ended Approach:**
    *   **Pattern:** "What are you in the mood for today – [Option A], [Option B], or something else?"
    *   **Example:** "What are you in the mood for today – a movie, a concert, or something else entirely?"
    *   **Why it's better:** It's inclusive ("what are *you* in the mood for"), gives clear options, and explicitly invites other ideas, reducing the pressure to choose *only* between your suggestions.

2.  **The Preference-Focused Approach:**
    *   **Pattern:** "Between [Option A] and [Option B], which sounds more appealing to you today?"
    *   **Example:** "Between watching a movie and going to a concert, which sounds more appealing to you today?"
    *   **Why it's better:** It directly asks about their preference, making it about what *they* would enjoy rather than which option is objectively "better."

3.  **The Benefit-Oriented & Suggestive Approach:**
    *   **Pattern:** "I was thinking we could [describe Option A experience], or if you prefer, we could [describe Option B experience]. What do you think?"
    *   **Example:** "I was thinking we could chill out with a movie today, or if you prefer, we could go for the excitement of a concert. What do you think?"
    *   **Why it's better:** It frames the options by their potential experiences/benefits, making them more attractive and giving the other person a clearer picture of what each entails. It also shows you've put some thought into it.

4.  **The Direct & Flexible Approach:**
    *   **Pattern:** "For today, would you prefer [Option A], or are you leaning more towards [Option B]? I'm open to either!"
    *   **Example:** "For today, would you prefer watching a movie, or are you leaning more towards a concert? I'm open to either!"
    *   **Why it's better:** It's clear, direct, uses "prefer" and "leaning towards" (softer than "better"), and crucially, your statement "I'm open to either!" makes it truly collaborative and less like you're pushing one option.

---

### **General Tips for Better Questioning:**

*   **Be Specific:** (Your original question was good here with "today.")
*   **Be Open-Ended:** Avoid questions that only lead to "yes" or "no" if you want more information.
*   **Focus on "You":** Frame questions around the other person's preference, feelings, or needs.
*   **Invite Alternatives:** Explicitly ask if they have other ideas.
*   **Provide Context (if helpful):** Briefly explain *why* you're asking or why you suggested certain options.
*   **Express Your Own Openness:** Let them know you're flexible.

---

**Would you like me to use one of these patterns, or a combination, whenever you ask a question in the future?** Please let me know your preference!

#### D- Flipped interaction Pattern
- I would like you to ask me questions to achieve X
- You should ask questions until condition Y is met or to achieve this goal (alternatively, forever)
- (Optional) ask me the questions one at a time, two at a time, ask me the first question, etc.


In [90]:
#TODO: Flipped interaction Pattern
prompt=""" Ask me several questions to find my age. You can ask until you get relevant informations to find the answer and always one question at a time.
Question:

example:
question:What are you studying?
answer: Masters
"""

response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

Okay, I understand. I will ask one question at a time, and we'll see what information you share!

Question 1: Are you currently in school, working, or retired?

--------------
## 2. Prompt Methods
#### 2.1- Zero-shot prompting

**Goal:** Implement Zero Shot Prompting and print the output of the model


In [91]:

# TODO: Implement Zero Shot Prompting and print the output of the model
prompt="""Add any 3 numbers of you liking and print the output
"""
response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

Okay! I'll choose the numbers 15, 20, and 5.

```python
# My chosen numbers
num1 = 15
num2 = 20
num3 = 5

# Calculate the sum
total = num1 + num2 + num3

# Print the output
print(f"The numbers I chose are: {num1}, {num2}, and {num3}")
print(f"Their sum is: {total}")
```

**Output:**

```
The numbers I chose are: 15, 20, and 5
Their sum is: 40
```


#### 2.2. Few-shot prompting

**Goal:** Provide **demonstrations** (input→output pairs) to steer format and tone.


In [92]:
#TODO: Implement Few Shot Prompting and print the output of the model
prompt="""Give a guide to how to cook Pasta

question1:how to start?
ans:boil water first

question2:should we add vegetables?
answer: yes

question3:should we add sauce?
answer:
"""
response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

question3:should we add sauce?
answer: **Absolutely! Sauce is a crucial component of almost all pasta dishes.** It adds flavor, moisture, and transforms plain pasta into a complete meal. You'll typically add your chosen sauce to the cooked and drained pasta, often mixing it together in a large pan over low heat for a minute or two to allow the flavors to meld. There's a vast array of sauces to choose from, like tomato-based (marinara, bolognese), cream-based (alfredo, carbonara), pesto, or olive oil and garlic.


#### 2.3. Chain-of-Thought (CoT)

**Goal:** Encourage step-by-step reasoning.  
Generate a chain of thought – series of intermediate reasoning steps- significantly improves the ability of LLMs to perform complex reasoning.



In [93]:
#TODO: Implement Chain of Thought Prompting and print the output of the model
prompt="""A train leaves City A at 9:00 AM traveling at 60 km/h. Another train leaves City B at 10:00 AM traveling towards City A at 80 km/h. The distance between City A and City B is 420 km. At what time will the two trains meet?

step-by-step reasoning:
"""

response = client.models.generate_content(model=MODEL_FAST, contents=prompt)
display(Markdown(response.text))

Here's a step-by-step reasoning to determine when the two trains will meet:

1.  **Calculate the head start of Train A:**
    *   Train A leaves at 9:00 AM. Train B leaves at 10:00 AM.
    *   This means Train A travels alone for 1 hour (from 9:00 AM to 10:00 AM).
    *   Distance covered by Train A in this hour = Speed × Time = 60 km/h × 1 h = 60 km.

2.  **Determine the remaining distance at 10:00 AM:**
    *   The total distance between City A and City B is 420 km.
    *   At 10:00 AM, Train A has already covered 60 km.
    *   The remaining distance between the two trains is 420 km - 60 km = 360 km.

3.  **Calculate the relative speed of the trains:**
    *   At 10:00 AM, both trains are moving towards each other.
    *   Relative speed = Speed of Train A + Speed of Train B
    *   Relative speed = 60 km/h + 80 km/h = 140 km/h.

4.  **Calculate the time it takes for them to meet from 10:00 AM onwards:**
    *   Time = Remaining Distance / Relative Speed
    *   Time = 360 km / 140 km/h
    *   Time = 36 / 14 hours
    *   Time = 18 / 7 hours

5.  **Convert the time (18/7 hours) into hours and minutes:**
    *   18 / 7 hours = 2 and 4/7 hours.
    *   2 hours is straightforward.
    *   To convert 4/7 of an hour into minutes: (4/7) × 60 minutes = 240 / 7 minutes.
    *   240 ÷ 7 ≈ 34.28 minutes.
    *   For practical purposes, this is approximately 2 hours and 34 minutes. (If you want to be super precise, it's 2 hours, 34 minutes, and 17 seconds).

6.  **Determine the final meeting time:**
    *   The trains started moving towards each other from 10:00 AM.
    *   They travel for an additional 2 hours and approximately 34 minutes.
    *   Meeting time = 10:00 AM + 2 hours 34 minutes = 12:34 PM.

The two trains will meet at **12:34 PM**.


#### 2.4 Self-Consistency (sampling multiple chains)

**Goal:** Improve reliability by sampling multiple reasoning paths and aggregating.


In [94]:

from collections import Counter

def solve_with_self_consistency(prompt, n=5, temperature=1.0, model=MODEL_FAST):
    answers = []
    for _ in range(n):
        resp = client.models.generate_content(
            model=model,
            contents=prompt,
            config=types.GenerateContentConfig(temperature=temperature)
        )
        text = resp.text.strip()
        # Naive final-answer extraction: last line or digits
        lines = [x for x in text.splitlines() if x.strip()]
        answers.append(lines[-1] if lines else text)
    tally = Counter(answers)
    return tally.most_common(1)[0], tally

sc_prompt = (
    "Tom has 10 apples. He gives 3 to Mary and 2 to John. "
    "How many apples remain? Think step by step, end with 'Final answer: <number>'."
)

winner, dist = solve_with_self_consistency(sc_prompt, n=5, temperature=0.9, model=MODEL_FAST)
print('Distribution:', dist)
print('Chosen:', winner)


Distribution: Counter({'Final answer: 5': 5})
Chosen: ('Final answer: 5', 5)



#### 2.5 Prompt chaining (sequential pipeline)

**Goal:** Decompose tasks. We’ll do: Extract → Expand → Synthesize.
- step1: Extract 3 key points from the text
- Step2: expand each point
- Step3 : Synthesize into one cohesive paragraph (<=120 words)

In [95]:

article = """
Network slicing enables multiple virtualized networks on shared infrastructure.
It tailors resources to use-cases like eMBB, URLLC, and mMTC.
Challenges include isolation, orchestration, and SLA enforcement.
"""

p1 = client.models.generate_content(
    model=MODEL_FAST,
    contents=f"Extract 3 key points from the text:\n{article}",
    config=types.GenerateContentConfig(temperature=0.4)
).text

p2 = client.models.generate_content(
    model=MODEL_FAST,
    contents=f"Expand each point (1-2 sentences each):\n{p1}",
    config=types.GenerateContentConfig(temperature=0.6)
).text

p3 = client.models.generate_content(
    model=MODEL_FAST,
    contents=f"Synthesize into one cohesive paragraph (<=120 words):\n{p2}",
    config=types.GenerateContentConfig(temperature=0.5)
).text

print("Key points:\n", p1, "\n\nExpanded:\n", p2, "\n\nSynthesis:\n", p3)


Key points:
 Here are 3 key points from the text:

1.  **Definition:** Network slicing enables the creation of multiple virtualized networks on a shared physical infrastructure.
2.  **Purpose:** It allows resources to be tailored and optimized for specific use cases, such as eMBB, URLLC, and mMTC.
3.  **Challenges:** Implementing network slicing involves overcoming challenges related to isolation, orchestration, and enforcing Service Level Agreements (SLAs). 

Expanded:
 Here are the expanded points:

1.  **Definition:** Network slicing allows operators to create several independent, end-to-end logical networks atop a single, underlying physical network infrastructure. Each "slice" functions as a dedicated network, customized with specific resources and functionalities for different services or applications.
2.  **Purpose:** The primary purpose of network slicing is to precisely allocate and configure network resources—such as bandwidth, latency, and reliability—to meet the diverse dem

#### 2.5.2 : Convert the **prompt chaining** into a function and add a **critique-and-revise** step.

In [96]:
# TODO
def prompt_chain(text, client, model=MODEL_FAST):
    # Step 1: Extract key points
    extract_prompt = f"Extract 3 key points from the following text:\n{text}"
    extract_resp = client.models.generate_content(model=model, contents=extract_prompt)
    key_points = extract_resp.text.strip().split("\n")

    # Step 2: Expand each point
    expanded_points = []
    for point in key_points:
        expand_prompt = f"Expand on this point in 2-3 sentences:\n{point}"
        expand_resp = client.models.generate_content(model=model, contents=expand_prompt)
        expanded_points.append(expand_resp.text.strip())

    # Step 3: Synthesize into one paragraph
    synth_prompt = "Combine the following points into one cohesive paragraph (<=120 words):\n" + "\n".join(expanded_points)
    synth_resp = client.models.generate_content(model=model, contents=synth_prompt)
    paragraph = synth_resp.text.strip()

    # Step 4: Critique and revise
    critique_prompt = f"Critique the following paragraph and revise it for clarity and conciseness:\n{paragraph}"
    critique_resp = client.models.generate_content(model=model, contents=critique_prompt)
    final_paragraph = critique_resp.text.strip()

    return final_paragraph

# Usage
text = "Your input text goes here."
result = prompt_chain(text, client)
display(Markdown(result))


ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'Resource exhausted. Please try again later. Please refer to https://cloud.google.com/vertex-ai/generative-ai/docs/error-code-429 for more details.', 'status': 'RESOURCE_EXHAUSTED'}}


#### 3. Structure control with JSON Response Schema

**Goal:** Force well-structured, machine-parseable outputs (e.g., for pipelines).  
We’ll ask the model to extract KPIs into a JSON object **validated by the SDK**.


In [ ]:

from pydantic import BaseModel

class KPIReport(BaseModel):
    kpi: str
    value: float
    unit: str
    interpretation: str

text = (
    "KPI: Latency. Value: 85 ms. In 5G eMBB, typical interactive threshold is 100 ms. "
    "Provide a one-sentence interpretation."
)

resp = client.models.generate_content(
    model=MODEL_FAST,
    contents=f"Extract a JSON report with fields (kpi, value, unit, interpretation) from: {text}",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=KPIReport,
        temperature=0.3,
    ),
)
print(resp.text)  # valid JSON per schema


#### give another example of structured output with a json response schema

In [ ]:
# TODO

from pydantic import BaseModel

class ArticleReport(BaseModel):
    title: str
    author: str
    date: str
    key_points: str
    summary:str

text = (
    "Title: AI Revolution in Healthcare. Author: Jane Doe. Date: 2025-10-23. "
    "The adoption of AI in hospitals has increased efficiency and reduced errors. "
    "However, ethical concerns regarding patient data privacy remain."
)

resp = client.models.generate_content(
    model=MODEL_FAST,
    contents=f"Extract a JSON report with fields (title, author, date, key_points, summary) from: {text}",
    config=types.GenerateContentConfig(
        response_mime_type="application/json",
        response_schema=ArticleReport,
        temperature=0.3,
    ),
)
print(resp.text)  # valid JSON per schema



#### 4. ReAct-style tool use (function calling)

We’ll register a simple tool and let the model decide when to call it.  
*(In production, you would call external APIs, search, databases, etc.)*


In [ ]:

# Demo tool: tiny facts DB
FACTS = {
    "instagram_acquirer": "Facebook acquired Instagram in 2012.",
    "facebook_ceo": "The CEO of Facebook (Meta) is Mark Zuckerberg."
}

def lookup_fact(key: str) -> str:
    """Return a tiny fact from the in-memory dict."""
    return FACTS.get(key, "Unknown.")

tool_config = types.ToolConfig(
    function_calling_config=types.FunctionCallingConfig(mode="ANY")
)

resp = client.models.generate_content(
    model=MODEL_FAST,
    contents="Who is the CEO of the company that acquired Instagram?",
    config=types.GenerateContentConfig(
        tools=[lookup_fact],
        tool_config=tool_config,
        # Optional: limit number of automatic function calls
        automatic_function_calling=types.AutomaticFunctionCallingConfig(maximum_remote_calls=2),
        temperature=0.3
    ),
)
pp(resp)

# Manual execution
result = lookup_fact("facebook_ceo")
print("Result:", result)




**Extend** the **ReAct** tool set with a math `calculator(a, b, op)` function and observe function calling.

In [ ]:
import json

# Generate content WITHOUT automatic function calling
calc_resp = client.models.generate_content(
    model=MODEL_FAST,
    contents="Call the calculator function with a=12, b=7, op='*'",
    config=types.GenerateContentConfig(
        tools=[calculator],
        temperature=0
    ),
)

# Parse the response manually
if calc_resp.candidates:
    candidate = calc_resp.candidates[0]

    # Some SDK versions return function_call as a dict inside content[0].text or content[0].function_call
    for part in candidate.content:
        func_call = getattr(part, "function_call", None)
        if func_call:
            name = func_call.name
            args = json.loads(func_call.arguments)
            print(f"Function '{name}' called with arguments {args}")

            # Execute the function
            if name == "calculator":
                result = calculator(**args)
                print("Result:", result)

# Directly call the calculator function
result = calculator(a=12, b=7, op='*')
print("Result:", result)
